## Higher Order Functions

  > - Higher Order Functions are functions that operate on complex data types such as arrays and maps
  > - They allow you to pass functions as arguments (such as lambda expressions), apply transformations and return arrays or maps 
  > - They are extremely useful for manipulating arrays without exploding them. 


#### Commonly used Higher Order Array Functions  
  - TRANSFORM
  - FILTER
  - EXISTS
  - AGGREGATE    

#### Syntax
-----------------------------------------------------------------------------
`<function_name> (array_column, lambda_expression)` 

_lambda_expression_: `element -> expression`


In [0]:
CREATE OR REPLACE TEMPORARY VIEW order_items AS
SELECT * FROM 
VALUES
  (1, array('smartphone', 'laptop', 'monitor')),
  (2, array('tablet', 'headphones', 'smartwatch')),
  (3, array('keyboard', 'mouse'))
AS orders(order_id, items);

In [0]:
select * from order_items

##### 1. Convert all the item names to be UPPERCASE (TRANSFORM Function)

In [0]:
select order_id,
       transform(items, x -> upper(x)) as upper_items
       from order_items

##### 2. Filter only items that contains the string 'smart' (FILTER Function)

In [0]:
select order_id,
       filter(items, x -> x like '%smart%') as smart_items
       from order_items

##### 3. Check to see whether the order includes any 'monitor' (EXISTS Function)

In [0]:
select order_id,
       exists(items, x -> x = 'monitor') as smart_items
       from order_items

### Array with more than one object

In [0]:
CREATE OR REPLACE TEMP VIEW order_items AS
SELECT * FROM VALUES
  (1, array(
        named_struct('name', 'smartphone', 'price', 699),
        named_struct('name', 'laptop', 'price', 1199),
        named_struct('name', 'monitor', 'price', 399)
    )),
  (2, array(
        named_struct('name', 'tablet', 'price', 599),
        named_struct('name', 'headphones', 'price', 199),
        named_struct('name', 'smartwatch', 'price', 299)
    )),
  (3, array(
        named_struct('name', 'keyboard', 'price', 89),
        named_struct('name', 'mouse', 'price', 59)
    ))
AS orders(order_id, items);


In [0]:
select * from order_items

##### 1. Convert all the item names to be UPPERCASE & Add 10% TAX to each item (TRANSFORM Function)

In [0]:
select order_id,
       transform(items, x -> named_struct(
                                          'name', upper(x.name),
                                          'price', round(x.price * 1.1, 2)
                                          )) items_with_tax
      from order_items

##### 2. Calculate the total order amount for each of the order (AGGREGATE Function)

In [0]:
select order_id,
       aggregate( items, 0, (acc,x) -> acc + x.price) as total_order_price
      from order_items

### Map Functions

A map is a collection of key-value pairs, like a dictionary  
`{'laptop': 1200, 'phone': 699}`


### Commonly used Higher Order Map Functions  
  - TRANSFORM_VALUES
  - TRANSFORM_KEYS
  - MAP_FILTER

#### Syntax
-----------------------------------------------------------------------------
`<function_name> (map_column, lambda_expression)`  
_lambda expression_: `(key, value) -> expression)`

In [0]:
CREATE OR REPLACE TEMP VIEW order_item_prices AS
SELECT * FROM VALUES
  (1, map('smartphone', 699, 'laptop', 1199, 'monitor', 399)),
  (2, map('tablet', 599, 'headphones', 199, 'smartwatch', 299)),
  (3, map('keyboard', 89, 'mouse', 59))
AS orders(order_id, item_prices);

In [0]:
select * from order_item_prices

##### 1. Convert all the item names to be UPPERCASE (TRANSFORM_KEYS Function)

In [0]:
select order_id,
        transform_keys(item_prices, (item, price) -> upper(item)) as items_upper_case
    from order_item_prices

##### 2. Apply 10% TAX to item prices (TRANSFORM_VALUES Function)

In [0]:
select order_id,
        transform_values(item_prices, (item, price) -> round(price * 1.1, 2)) as prices_with_tax
    from order_item_prices

##### 3. Filter only items with price above $500 (MAP_FILTER Function)

In [0]:
select order_id,
        map_filter(item_prices, (item, price) -> price > 500) as premium_items
    from order_item_prices